In [1]:
import polars
import hvplot.polars

import holoviews as hv

from polars import col

HEIGHT = 600
WIDTH = 1200
TICKER_COUNT = 100

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
amd = polars.read_csv("../tests/data/AMD.csv", try_parse_dates=True).tail(TICKER_COUNT)
amd.head()

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Time Zone
date,f64,f64,f64,f64,i64,f64,f64,str
2024-03-18,193.289993,193.5,188.050003,190.649994,59306900,0.0,0.0,"""EDT"""
2024-03-19,183.160004,185.610001,177.360001,181.419998,83509500,0.0,0.0,"""EDT"""
2024-03-20,182.029999,182.679993,174.0,179.729996,81773100,0.0,0.0,"""EDT"""
2024-03-21,185.970001,187.690002,177.660004,178.679993,75265600,0.0,0.0,"""EDT"""
2024-03-22,177.210007,180.759995,175.050003,179.649994,57832100,0.0,0.0,"""EDT"""


In [3]:
import indicatory.oscillators as osc

In [4]:
short = 5
long = 20
ppo = osc.percentage_price_oscillator(amd, short_window_size=short, long_window_size=long)
simple_rsi = osc.simple_relative_strength_index(amd)
rsi = osc.relative_strength_index(amd)
so = osc.stochastic_oscillator(amd)

In [5]:
ohlc = amd.tail(TICKER_COUNT - long).hvplot.ohlc(x="Date", y=["Open", "High", "Low", "Close"], grid=True, width=WIDTH, height=HEIGHT)

ppo_plot = ppo.tail(TICKER_COUNT - long).hvplot.scatter(x="Date", y=osc.ppo_column(short, long, "Close"), grid=True, width=WIDTH, height=int(HEIGHT / 3))
hline = hv.HLine(0).options(color='black', line_width=1)

hv.Layout(ohlc + ppo_plot * hline).cols(1) 

:Layout
   .Overlay.I  :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Overlay.II :Overlay
      .Scatter.I :Scatter   [Date]   (PPO 5/20 Close)
      .HLine.I   :HLine   [x,y]

In [6]:
overbought_threshold = hv.HLine(70).options(line_width=1, color="red")
oversold_threshold = hv.HLine(30).opts(line_width=1, color="red")

simple_rsi_plot = simple_rsi.tail(TICKER_COUNT - long).hvplot.scatter(
    x="Date", 
    y=osc.rsi_column(10), 
    title="Simple RSI",
    grid=True, 
    width=WIDTH, 
    height=int(HEIGHT / 3)
)
rsi_plot = rsi.tail(TICKER_COUNT - long).hvplot.scatter(
    x="Date", 
    y=osc.rsi_column(10), 
    title="RSI",
    color='darkblue', 
    grid=True, 
    width=WIDTH, 
    height=int(HEIGHT / 3)
)

hv.Layout(ohlc + simple_rsi_plot * overbought_threshold * oversold_threshold + rsi_plot * overbought_threshold * oversold_threshold).cols(1)

:Layout
   .Overlay.I   :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Overlay.II  :Overlay
      .Scatter.I :Scatter   [Date]   (RSI 10)
      .HLine.I   :HLine   [x,y]
      .HLine.II  :HLine   [x,y]
   .Overlay.III :Overlay
      .Scatter.I :Scatter   [Date]   (RSI 10)
      .HLine.I   :HLine   [x,y]
      .HLine.II  :HLine   [x,y]

In [9]:
so_fast_plot = so.tail(TICKER_COUNT - long).hvplot.step(
    x="Date", 
    y=[osc.fast_k_percent(5), osc.fast_d_percent(5)],
    title="Stochastic Oscillators - Fast",
    color=["green", "orange"],
    grid=True, 
    width=WIDTH, 
    height=int(HEIGHT / 3)
)
so_slow_plot = so.tail(TICKER_COUNT - long).hvplot.line(
    x="Date", 
    y=[osc.slow_k_percent(5), osc.slow_d_percent(5)], 
    title="Stochastic Oscillators - Slow",
    color=["blue", "purple"],
    grid=True, 
    width=WIDTH, 
    height=int(HEIGHT / 3)
)

hv.Layout(ohlc + so_fast_plot * overbought_threshold * oversold_threshold + so_slow_plot * overbought_threshold * oversold_threshold).cols(1)

ValueError: 'Fast %K 5' is not in list